# Boosting

Boosting are sequential ensemble methods that train predictors based on previous predictors. The two most common types are AdaBoost and Gradient Boosting.

## AdaBoost (Adaptive Boosting)

AdaBoost works by trying to pay more attention to instances that are underfit giving them higher weights. AdaBoost works by training a sequence of predictors where the weighted majority vote gives the final prediction. The weight for each predictor is called the *predictor rate* is calculated from the *weighted error rate*:

$$ r_j = \frac{\underset{\hat{y}_i=y_i}{\sum_{i=1}^m w_i}}{\sum_{i=1}^m w_i} $$

$$\alpha_j = \eta \log \frac{1-r_j}{r_j} $$

W is usually initialised as 1/m. To update the weights incorrect predictions are updated with $e^\alpha_j$ and then normalised. To select the instances for the next predictor to train on it is done using roulette wheel selection based on the weights of each instance.

In [4]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
import numpy as np
import random
np.random.seed(42)

X, y = make_moons(n_samples=500, noise=0.3, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [14]:
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

dtc_clf = DecisionTreeClassifier(max_depth=1)

In [6]:
eta = 0.5
iterations = 20
total = np.array([0.0 for i in range(len(X_test))])
current_X = X_train
current_y = y_train
for i in range(iterations):
    dtc_clf.fit(current_X, current_y)
    y_pred = dtc_clf.predict(current_X) 

    weights = np.ones((len(X_train),1)).flatten() * 1/len(X_train)
    weighted_error = ((y_pred != current_y) * weights).sum()
    if weighted_error == 0:
        weighted_error = np.amin(weights)/10
    if weighted_error == 1:
        weighted_error = 1-np.amin(weights)/10

    alpha = eta * np.log((1-weighted_error)/weighted_error)
    preds = dtc_clf.predict(X_test)
    preds = np.where(preds==0, -1, preds)
    total += preds * alpha

    weights *= np.exp(np.where((y_pred != current_y)==0, -1, (y_pred != current_y))*alpha)
    weights = weights/weights.sum()

    current_X = np.empty((0,2))
    current_y = np.array([])
    for k in range(len(X_train)):
        n = random.uniform(0,1)
        temp_sum = 0
        for m in range(len(weights)):
            if temp_sum < n and n < (temp_sum + weights[m]):
                current_X = np.vstack((current_X, X_train[m]))
                current_y = np.hstack((current_y, y_train[m]))
                break
            temp_sum += weights[m]
total = (total>=0)
np.count_nonzero(total == y_test)/len(X_test)

0.824

## Gradient Boosting

Similarly to AdaBoost gradient boost will add many predictors sequentially. However it does this by fitting predictors on the residuals of the previous predictors and adding the results:

In [22]:
X = 2 * np.random.rand(100,1)
y = 4 + 4 * X + 2 * X**2 + np.random.randn(100,1)

data = np.hstack((X,y))
temp = data[np.argsort(data[:,0])]
X = temp[:,0].reshape((-1,1))
y = temp[:,1]

In [30]:
dtr_reg1 = DecisionTreeRegressor(max_depth=2)
dtr_reg1.fit(X,y)
y_pred = dtr_reg1.predict(X)
print("RMSE Before boosting:")
print(np.sqrt(sum((y_pred - y)**2)))

y2 = y - dtr_reg1.predict(X)
dtr_reg2 = DecisionTreeRegressor(max_depth=2)
dtr_reg2.fit(X,y2)

y3 = y - dtr_reg2.predict(X)
dtr_reg3 = DecisionTreeRegressor(max_depth=2)
dtr_reg3.fit(X,y3)

y_pred = sum(dtr.predict(X) for dtr in (dtr_reg1, dtr_reg2, dtr_reg3))/2

print("")
print("RMSE After Boosting")
print(np.sqrt(sum((y_pred - y)**2)))

RMSE Before boosting:
13.531119142765169

RMSE After Boosting
11.93704629367871


The above code implements gradient boosting succesfully. A common method in gradient boosting is to use a shrinkage parameter where each tree becomes sequentially less important. 